In [1]:
import json
from datetime import datetime
from app.utils.vercel_kv import KV
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage, BaseMessage
from langchain_openai import ChatOpenAI
from app.models import User, Message

kv = KV()
model = ChatOpenAI(model_name="gpt-4o")

In [ ]:
system_prompts = [
    "You are an Fitness Coach that helps the user loose weight. This is your first conversation with the user. Keep your answers short and simple and try to find out if the user is already tracking their steps. After that, try to learn about the first name, age, gender and height of the user. ",
]

In [10]:
def get_user(wa_id):
    user_object = kv.get(wa_id)
    if user_object is None or user_object == "":
        print("User is None")
        user = User(**user_object)
    else:
        print("User is not None")
        user = User()
        user.messages = []
    return user

def process_message(message, wa_id):
    user = get_user(wa_id)
        
    sys = Message(base_message=SystemMessage(system_prompts[0]))
    hum = Message(time=datetime.now(), base_message=HumanMessage(message))
    
    messages = [sys, hum]
    base_messages = [message.base_message for message in messages]
    reply = Message(time=datetime.now(), base_message=model.invoke(base_messages))
    
    messages.append(reply)
    user.messages = messages
    
    kv.set(wa_id, user.dict())
    # print(kv.get(wa_id))
    print("User: ", message)
    print("FitChat: ", reply.base_message.content)
    
process_message("Hiii", "+41799506553")

User is not None
User:  Hiii
FitChat:  Hi there! Are you currently tracking your steps?


In [5]:
test = kv.get("+41799506553")
print(test)

{"paid": false, "user_data": {"first_name": null, "age": null, "gender": null, "height": null}, "activity_log": null, "messages": [{"time": "2024-07-05T17:43:32.336889", "base_message": {"content": "You are an Fitness Coach that helps the user loose weight. This is your first conversation with the user. Keep your answers short and simple and try to find out if the user is already tracking their steps. After that, try to learn about the first name, age, gender and height of the user. ", "additional_kwargs": {}, "response_metadata": {}, "type": "system", "name": null, "id": null}}, {"time": "2024-07-05T17:43:32.336911", "base_message": {"content": "Hiii", "additional_kwargs": {}, "response_metadata": {}, "type": "human", "name": null, "id": null, "example": false}}, {"time": "2024-07-05T17:43:32.336943", "base_message": {"content": "Hello! Welcome to your fitness journey. Are you currently tracking your daily steps?", "additional_kwargs": {}, "response_metadata": {"token_usage": {"comple